In [81]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Introduction à Pandas

la question des `astype` : 
cette fonction change le type d'objets dans une `pd.Series`.

In [82]:
s = pd.Series([4,5,6], dtype=int)
print(f"les éléments de s sont de type {type(s[0])}")
# On change le type à string : 
s = s.astype(str)
print(f"les éléments de s sont de type {type(s[0])}")

les éléments de s sont de type <class 'numpy.int64'>
les éléments de s sont de type <class 'str'>


## Exercice préliminaire: importer un CSV

In [83]:
import requests

url = "https://www.insee.fr/fr/statistiques/fichier/6800675/v_commune_2023.csv"
url_backup = "https://minio.lab.sspcloud.fr/lgaliana/data/python-ENSAE/cog_2023.csv"

try:
    response = requests.get(url)
except requests.exceptions.RequestException as e:
    print(f"Error : {e}")
    response = requests.get(url_backup)

# Only download if one of the request succeeded
if response.status_code == 200:
    with open("cog_2023.csv", "wb") as file:
        file.write(response.content)

charger un fichier dans le même dossier :

```txt
folder
- cog_2023.csv
- notebook.ipynb
```

In [84]:
df = pd.read_csv("cog_2023.csv")

Charger un fichier dans un sous-dossier

```txt
folder
- data
    - cog_2023.csv
- notebook.ipynb
```

```python
df = pd.read_csv("./data/cog_2023.csv)
```

Charger un fichier qui est dans un dossier parent

```txt
folder
- codes
    - notebook.ipynb
- cog_2023.csv
```

```python
df = pd.read_csv("../cog_2023.csv")
```


Charger un fichier qui est dans un dossier voisin

```txt
folder
- data
    - cog_2023.csv
- codes
    - notebook.ipynb
```

```python
df = pd.read_csv("../data/cog_2023.csv")
```

## Exercice 1

1.  Importer les données de l’Ademe à l’aide du package `Pandas` et de la commande consacrée pour l’import de csv. Nommer le `DataFrame` obtenu `emissions`[1].

In [85]:
df = pd.read_csv("cog_2023.csv")

2.  Utiliser les méthodes adéquates afin d’afficher pour les 10 premières valeurs, les 15 dernières et un échantillon aléatoire de 10 valeurs grâce aux méthodes adéquates du *package* `Pandas`.

In [86]:
df_10_first = df.head(10)
print(len(df_10_first))
df_15_last = df.tail(15)
print(len(df_15_last))
df_10_random = df.sample(n = 10)
print(len(df_10_random))

10
15
10


3.  Tirer 5 pourcents de l’échantillon sans remise.

In [87]:
df_5_percent = df.sample(frac = 0.05)
print(f"{len(df_5_percent)} (len(df) = {len(df)})")

1878 (len(df) = 37563)


4.  Ne conserver que les 10 premières lignes et tirer aléatoirement dans celles-ci pour obtenir un DataFrame de 100 données.

In [88]:
df_10_first.sample(n = 100, replace = True)

,TYPECOM,COM,REG,DEP,CTCD,ARR,TNCC,NCC,NCCENR,LIBELLE,CAN,COMPARENT
0,COM,01001,84.0,01,01D,012,5,ABERGEMENT CLEMENCIAT,Abergement-Clémenciat,L'Abergement-Clémenciat,0108,NaN
9,COM,01011,84.0,01,01D,014,1,APREMONT,Apremont,Apremont,0114,NaN
2,COM,01004,84.0,01,01D,011,1,AMBERIEU EN BUGEY,Ambérieu-en-Bugey,Ambérieu-en-Bugey,0101,NaN
6,COM,01008,84.0,01,01D,011,1,AMBUTRIX,Ambutrix,Ambutrix,0101,NaN
3,COM,01005,84.0,01,01D,012,1,AMBERIEUX EN DOMBES,Ambérieux-en-Dombes,Ambérieux-en-Dombes,0122,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
8,COM,01010,84.0,01,01D,011,1,ANGLEFORT,Anglefort,Anglefort,0110,NaN
7,COM,01009,84.0,01,01D,011,1,ANDERT ET CONDON,Andert-et-Condon,Andert-et-Condon,0104,NaN
8,COM,01010,84.0,01,01D,011,1,ANGLEFORT,Anglefort,Anglefort,0110,NaN
6,COM,01008,84.0,01,01D,011,1,AMBUTRIX,Ambutrix,Ambutrix,0101,NaN


In [89]:
# Possible de chainer les commandes
_ = df.head(10).sample(n = 100, replace = True)

5.  Faire 100 tirages à partir des 6 premières lignes avec une probabilité de 1/2 pour la première observation et une probabilité uniforme pour les autres.

In [90]:
_ = df.head(6).sample(n = 100, weights = [0.5, 0.1, 0.1, 0.1, 0.1, 0.1], replace = True)
# Autre syntaxe : 
weights = [0.5] + [0.1] * 5
print(weights)
_ = df.head(6).sample(n = 100, weights = weights, replace = True)

[0.5, 0.1, 0.1, 0.1, 0.1, 0.1]


In [91]:
# Généralisation
N_in = 6
N_out = 100
weights = [0.5] + [0.5 / (N_in - 1)] * (N_in - 1)
_ = df.head(N_in).sample(n = N_out, weights = weights, replace = True)

## Exercice 2

1.  Créer un *dataframe* `emissions_copy` ne conservant que les colonnes `INSEE commune`, `Commune`, `Autres transports` et `Autres transports international`

In [92]:
colonnes_a_conserver = [
    'INSEE commune', 
    'Commune', 
    'Autres transports',
    'Autres transports international'
]
url = "https://koumoul.com/s/data-fair/api/v1/datasets/igt-pouvoir-de-rechauffement-global/convert"
emissions = pd.read_csv(url, usecols=colonnes_a_conserver)
# Possibilité de charger le tableau puis ensuite de sélectionner les colonnes.
# Pour les grands fichiers csv, privilégier la solution précédente

# df = pd.read_csv("./cog_2023.csv").loc[:,colonnes_a_conserver]

2. Comme les noms de variables sont peu pratiques, les renommer de la manière suivante :

-   `INSEE commune` $\to$ `code_insee`
-   `Autres transports` $\to$ `transports`
-   `Autres transports international` $\to$ `transports_international`

In [93]:
emissions = emissions.rename({
    "INSEE commune"                   : "code_insee",
    "Autres transports"               : "transports",
    "Autres transports international" : "transports_international"
}, axis = 1)

vérifions le résultat

In [94]:
emissions.head(1)

,code_insee,Commune,transports,transports_international
0,01001,L'ABERGEMENT-CLEMENCIAT,NaN,NaN


3.  On propose, pour simplifier, de remplacer les valeurs manquantes (`NA`) par la valeur 0. Utiliser la méthode [`fillna`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.fillna.html) pour transformer les valeurs manquantes en 0.

Visualisons les valeurs NaN

In [95]:
emissions.head()

,code_insee,Commune,transports,transports_international
0,01001,L'ABERGEMENT-CLEMENCIAT,NaN,NaN
1,01002,L'ABERGEMENT-DE-VAREY,NaN,NaN
2,01004,AMBERIEU-EN-BUGEY,212.577908,NaN
3,01005,AMBERIEUX-EN-DOMBES,NaN,NaN
4,01006,AMBLEON,NaN,NaN


In [96]:
emissions = emissions.fillna(0)

Vérifions le résultat

In [97]:
emissions.head()

,code_insee,Commune,transports,transports_international
0,01001,L'ABERGEMENT-CLEMENCIAT,0.000000,0.0
1,01002,L'ABERGEMENT-DE-VAREY,0.000000,0.0
2,01004,AMBERIEU-EN-BUGEY,212.577908,0.0
3,01005,AMBERIEUX-EN-DOMBES,0.000000,0.0
4,01006,AMBLEON,0.000000,0.0


4.  Créer les variables suivantes :

-   `dep`: le département. Celui-ci peut être créé grâce aux deux premiers caractères de `code_insee` en 
-   `transports_total`: les émissions du secteur transports (somme des deux variables)

In [98]:
# Colonne dep
## Solution str 
emissions["dep"] = emissions["code_insee"].str[:2]

## Solution apply
def retrieve_dep(code_insee : str):
    return code_insee[:2]
emissions["dep"] = emissions["code_insee"].apply(retrieve_dep)

# Colonne transport total
## Solution simple
emissions["transports_total"] = emissions["transports"].astype(float) + \
    emissions["transports_international"].astype(float)
# ici j'utilise le "astype" pour m'assurer que les données sont bien des floats 
# et ne pas introduire d'erreur

## Solution complexe (a ne pas utiliser dans ce cas là)
def compute_transport_total(row : dict) -> dict:
    # ici il faut ajouter un attribut à la colonne et pas simplement renvoyer 
    # une valeur
    row["transports_total"] = float(row["transports"]) +\
                              float(row["transports_international"])
    return row
emissions = emissions.apply(compute_transport_total, axis = 1)

In [99]:
emissions.head()

,code_insee,Commune,transports,transports_international,dep,transports_total
0,01001,L'ABERGEMENT-CLEMENCIAT,0.000000,0.0,01,0.000000
1,01002,L'ABERGEMENT-DE-VAREY,0.000000,0.0,01,0.000000
2,01004,AMBERIEU-EN-BUGEY,212.577908,0.0,01,212.577908
3,01005,AMBERIEUX-EN-DOMBES,0.000000,0.0,01,0.000000
4,01006,AMBLEON,0.000000,0.0,01,0.000000


5. Ordonner les données du plus gros pollueur au plus petit puis ordonner les données du plus gros pollueur au plus petit par département (du 01 au 95).

Vérifions d'abord à quoi ressemblent les données de `emissions["dep"]`:

In [100]:
emissions["dep"].unique()

array(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11',
       '12', '13', '14', '15', '16', '17', '18', '19', '21', '22', '23',
       '24', '25', '26', '27', '28', '29', '2A', '2B', '30', '31', '32',
       '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43',
       '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54',
       '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65',
       '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76',
       '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87',
       '88', '89', '90', '91', '92', '93', '94', '95'], dtype=object)

Toutes les valeurs sont des strings, vérifions que la fonction `sort_values` fonctionne comme on le souhaite, notamment avec la question des départements "2A" et "2B".

On note que les départements 2a et 2b remplacent le département 20 (qui n'existe pas).

In [101]:
s = pd.Series(["01", "2A", "2A", "2B", "41", "04","99","88", "19", "21", "15", "25"])\
    .sample(frac=1) # Shuffle
s

4     41
3     2B
7     88
2     2A
0     01
8     19
11    25
9     21
10    15
6     99
5     04
1     2A
dtype: object

In [102]:
s.sort_values()

0     01
5     04
10    15
8     19
9     21
11    25
2     2A
1     2A
3     2B
4     41
7     88
6     99
dtype: object

On voit donc que la méthode classique de `sort_values` ne peut pas nous aider dans ce cas. Pour l'utiliser, il faut créer une autre colonne `"dep-float"` qui gère correctement les départements de corse.

In [103]:
def dep_float(dep: str):
    try:
        # Tente de retourner la valeur float(dep)
        # Ce qui fonctionnera dans tous les cas sauf pour la corse
        return float(dep)
    except:
        # Dans le cas où ça ne marche pas (ie les cas de corse), on distingue les 
        # cas
        if dep == "2A": return 20
        elif dep == "2B": return 20.5
        else:
            # Rien ne devrait tomber dans cette catégorie, mais dans le doute, 
            # on prépare un moyen de les repérer
            print(dep)
            return np.nan
emissions["dep-float"] = emissions["dep"].apply(dep_float)

Nous n'avons pas eu d'erreur, donc notre fonction a sans doute fonctionné.
Vérifions :

In [104]:
emissions["dep-float"].unique()

array([ 1. ,  2. ,  3. ,  4. ,  5. ,  6. ,  7. ,  8. ,  9. , 10. , 11. ,
       12. , 13. , 14. , 15. , 16. , 17. , 18. , 19. , 21. , 22. , 23. ,
       24. , 25. , 26. , 27. , 28. , 29. , 20. , 20.5, 30. , 31. , 32. ,
       33. , 34. , 35. , 36. , 37. , 38. , 39. , 40. , 41. , 42. , 43. ,
       44. , 45. , 46. , 47. , 48. , 49. , 50. , 51. , 52. , 53. , 54. ,
       55. , 56. , 57. , 58. , 59. , 60. , 61. , 62. , 63. , 64. , 65. ,
       66. , 67. , 68. , 69. , 70. , 71. , 72. , 73. , 74. , 75. , 76. ,
       77. , 78. , 79. , 80. , 81. , 82. , 83. , 84. , 85. , 86. , 87. ,
       88. , 89. , 90. , 91. , 92. , 93. , 94. , 95. ])

on retrouve bien la valeur `20`, `20.5` ainsi que toutes les valeurs entre 1 et 95

In [105]:
# On peut maintenant ordonner le tableau à l'aide de la fonction sort_values
emissions = emissions.sort_values(by = ["dep-float","transports_total"], 
                                  ascending = [True, False])
emissions.head(15)

,code_insee,Commune,transports,transports_international,dep,transports_total,dep-float
160,01179,GRIEGES,617.281080,21.826353,01,639.107433,1.0
14,01016,ARBIGNY,382.957621,22.094924,01,405.052545,1.0
382,01427,TREVOUX,361.901404,20.880076,01,382.781480,1.0
307,01348,SAINT-DIDIER-SUR-CHALARONNE,363.104704,19.215886,01,382.320590,1.0
358,01402,SERMOYER,340.973268,19.672617,01,360.645885,1.0
234,01263,MONTMERLE-SUR-SAONE,315.060997,18.177596,01,333.238592,1.0
143,01157,FAREINS,254.995208,14.712071,01,269.707279,1.0
113,01123,CORMORANCHE-SUR-SAONE,253.233559,11.712017,01,264.945577,1.0
285,01322,REYRIEUX,223.437136,11.741258,01,235.178394,1.0
299,01339,SAINT-BERNARD,222.318063,12.826748,01,235.144811,1.0


6. Ne conserver que les communes appartenant aux départements 13 ou 31. Ordonner ces communes du plus gros pollueur au plus petit.

In [106]:
emissions_hg_bdr = emissions.\
    loc[emissions['dep'].isin(["13","31"])].\
    sort_values("transports", ascending = False)

7.  Calculer les émissions totales par secteur. Calculer la part de chaque secteur dans les émissions totales. Transformer en tonnes les volumes avant d’afficher les résultats

In [107]:
# ATTENTION, pour cette question il faut re-télécharger le jeu de données
df = pd.read_csv(url)

emissions_totales = (
  pd.DataFrame(
    df.sum(numeric_only = True),
    columns = ["emissions"]
  )
  .reset_index(names = "secteur")
)

emissions_totales['emissions (%)'] = (
  100*emissions_totales['emissions']/emissions_totales['emissions'].sum()
)
emissions_totales["emissions"] = emissions_totales["emissions"].div(1000)
(emissions_totales
  .sort_values("emissions", ascending = False)
  .round()
  .head(5)
)

emissions_totales

,secteur,emissions,emissions (%)
0,Agriculture,87909.693748,16.548342
1,Autres transports,6535.446083,1.230249
2,Autres transports international,22238.569280,4.186244
3,CO2 biomasse hors-total,63519.310716,11.957035
4,Déchets,14703.580140,2.767839
5,Energie,22852.033998,4.301725
6,Industrie hors-énergie,83573.677442,15.732120
7,Résidentiel,63841.398385,12.017666
8,Routier,126493.163531,23.811392
9,Tertiaire,39562.729439,7.447388


8.  Calculer pour chaque commune les émissions totales après avoir imputé les valeurs manquantes à 0. Garder les 100 communes les plus émettrices. Calculer la part de chaque secteur dans cette émission. Comprendre les facteurs pouvant expliquer ce classement.

In [108]:
emissions_totales = emissions.\
    fillna(0).\
    sum(numeric_only=True, axis = 1).\
    sort_values(ascending = False).\
    head(100)

emissions_top = emissions.iloc[emissions_totales.index].fillna(0)

emissions_top

,code_insee,Commune,transports,transports_international,dep,transports_total,dep-float
30872,77046,BOULANCOURT,0.000000,0.0,77,0.000000,77.0
30857,77031,BERNAY-VILBERT,0.000000,0.0,77,0.000000,77.0
30876,77050,BRANSLES,0.000000,0.0,77,0.000000,77.0
30259,76163,CATENAY,0.000000,0.0,76,0.000000,76.0
35388,91332,LEUDEVILLE,0.000000,0.0,91,0.000000,91.0
...,...,...,...,...,...,...,...
26791,66174,SAINT-FELIU-D'AVALL,82.726135,0.0,66,82.726135,66.0
12805,33140,CREON,0.000000,0.0,33,0.000000,33.0
18844,51049,BERGERES-LES-VERTUS,125.843555,0.0,51,125.843555,51.0
30258,76162,LE CATELIER,0.000000,0.0,76,0.000000,76.0
